In [1]:
# the problem: load max(M) in order to fill max(space)
#IMPORT
from qiskit import BasicAer
from qiskit_aqua import QuantumInstance
from qiskit_aqua import Operator, run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import aircargo
from qiskit_aqua.algorithms import VQE, QAOA, ExactEigensolver
from qiskit_aqua.components.optimizers import COBYLA
from qiskit_aqua.components.variational_forms import RY
import numpy as np

#set for real device
#device = 'ibmq_16_melbourne'
from qiskit import IBMQ
IBMQ.load_accounts()
#backend = IBMQ.get_backend(device)

# define the problem
# variables
# m_max - maxumum payload; scalar
# n - number of mass elements to be loaded - boxes
# N - number of spaces available for boxes; an unit space is = 1/2 of original_space
# M - mass vector n x 1
# K - space took by each mass boxes; vector n x 1
# p - penalty

m_max=6
n=5
N=4  # as defined into the original problem there are 2 storage locations; here we use a location = 1/2 of original
p = 1/5.0
M=np.array([2.1, 3.2, 0.9, 2.8, 1.2])
#M=np.array([2.1/6, 3.2/6, 0.9/6, 0.8/6, 1.2/6]) # in this case is reported to m_max; m_max becomes = 1
K=np.array([0.5,0.5,0.25,0.25,0.25])
print('M',M)
print('K',K)
e=np.ones(n)
E = np.matmul(np.asmatrix(e).T, np.asmatrix(e))

qubitOp, offset = aircargo.get_aircargo_qubitops_01(M, K, m_max, p)
#qubitOp, offset = aircargo.get_aircargo_qubitops_fiacco(M, K, m_max, p)
algo_input = EnergyInput(qubitOp)
print(offset, qubitOp)

#prepare some printing format
def index_to_selection(i, n):
    s = "{0:b}".format(i).rjust(n)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(n))])
    return x

def print_result(result):
    selection = aircargo.sample_most_likely(result['eigvecs'][0])
    value = aircargo.aircargo_value_01(selection, M, K, m_max, p)
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))

    probabilities = np.abs(result['eigvecs'][0])**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, n)
        value = aircargo.aircargo_value_01(x, M, K, m_max, p)    
        probability = probabilities[i]
        if value>=0: print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

#exact eigensolver - as reference
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
result = exact_eigensolver.run()
print('\n','exact_eigensolver')
print_result(result)

#VQE
backend = BasicAer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=250)
ry = RY(qubitOp.num_qubits, depth=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla, 'matrix')
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_mapper=seed)

result = vqe.run(quantum_instance)
print('\n','VQE')
print_result(result)

#QAOA
#backend = BasicAer.get_backend('statevector_simulator')
#seed = 50

#cobyla = COBYLA()
#cobyla.set_options(maxiter=250)
#qaoa = QAOA(qubitOp, cobyla, 3, 'matrix')
#qaoa.random_seed = seed

#quantum_instance = QuantumInstance(backend=backend, seed=seed, seed_mapper=seed)

#result = qaoa.run(quantum_instance)
#print('\n','QAOA')
#print_result(result)


M [2.1 3.2 0.9 2.8 1.2]
K [0.5  0.5  0.25 0.25 0.25]
14.575500000000003 Representation: paulis, qubits: 5, size: 15

 exact_eigensolver
Optimal: selection [0 1 0 1 0], value 5.9875

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 1 0 1 0]	5.9875		1.0000
[1 1 1 1 1]	6.5595		0.0000
[0 1 1 1 1]	7.2055		0.0000
[0 1 0 0 0]	1.5820		0.0000
[1 1 0 0 0]	5.2020		0.0000
[1 0 1 0 0]	1.1875		0.0000
[0 1 1 0 0]	3.3655		0.0000
[1 1 1 0 0]	6.1795		0.0000
[0 0 0 1 0]	0.6395		0.0000
[1 0 0 1 0]	4.6455		0.0000
[1 1 0 1 0]	7.2055		0.0000
[0 0 1 1 0]	2.5920		0.0000
[1 0 1 1 0]	5.7920		0.0000
[0 1 1 1 0]	6.7380		0.0000
[1 1 1 1 0]	7.1500		0.0000
[1 0 0 0 1]	1.8295		0.0000
[0 1 0 0 1]	3.8755		0.0000
[1 1 0 0 1]	6.4375		0.0000
[1 0 1 0 1]	3.5520		0.0000
[0 1 1 0 1]	5.2020		0.0000
[1 1 1 0 1]	6.9580		0.0000
[0 0 0 1 1]	3.1500		0.0000
[1 0 0 1 1]	6.0980		0.0000
[0 1 0 1 1]	6.9120		0.0000
[1 1 0 1 1]	7.0720		0.0000
[0 0 1 1 1